In [1]:
import random
import time
import re
from datetime import date
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', None)

## 生成每个页面的链接

通过链家网的搜索结果可以看出，二手房源数量有3万多个，由于网站最多只显示100个页面的房源，每个页面30个房源，所以无法采集到厦门所有的二手房数据，这里采用分区域采集，思明湖里集美海沧，这4个区的房源数量都超过3000个，但只能采集到3000个，翔安同安的房源都可以采集到。

In [2]:
def make_urls(location, n):
    urls = []
    for i in range(1, n+1):
        url = 'https://xm.lianjia.com/chengjiao/{}/pg{}/'.format(location, i)
        urls.append(url)
    return urls


siming = make_urls('siming', 100)
huli = make_urls('huli', 72)
jimei = make_urls('jimei', 74)
haicang = make_urls('haicang', 73)
xiangan = make_urls('xiangan', 22)
tongan = make_urls('tongan', 8)

xm_urls_list = siming + huli + jimei + haicang + xiangan + tongan
len(xm_urls_list)

349

## 处理User-Agent和Cookie

In [3]:
# 定义函数，用来处理User-Agent和Cookie
def ua_ck():
    '''
    网站需要登录才能采集，需要从Network--Doc里复制User-Agent和Cookie，Cookie要转化为字典
    '''

    user_agent = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'}

    cookies = 'select_city=350200; lianjia_uuid=c6bb527e-c73e-4c18-80a2-0e6c701d4f40; _smt_uid=5f649daa.382e09bd; UM_distinctid=174a107e3f6810-0ca2d4a8dbcaa8-333769-125f51-174a107e3f7c3e; _ga=GA1.2.1661642603.1600429486; _gid=GA1.2.527914271.1600429486; Hm_lvt_9152f8221cb6243a53c83b956842be8a=1600429483,1600476524; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%22174a107e556cb9-037ed71a329a2c-333769-1204049-174a107e557d11%22%2C%22%24device_id%22%3A%22174a107e556cb9-037ed71a329a2c-333769-1204049-174a107e557d11%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E7%9B%B4%E6%8E%A5%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22%22%2C%22%24latest_referrer_host%22%3A%22%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%7D%7D; lianjia_ssid=0857e41c-7ca8-24c2-7d18-e2b4b02e4d13; User-Realip=140.243.198.151; CNZZDATA1254525948=1557469094-1600424880-%7C1600525641; CNZZDATA1255847100=75444696-1600427554-%7C1600525837; CNZZDATA1255633284=1437618958-1600429218-%7C1600526300; CNZZDATA1255604082=1475741897-1600429003-%7C1600526307; _gat=1; _gat_global=1; _gat_new_global=1; _gat_dianpu_agent=1; Hm_lpvt_9152f8221cb6243a53c83b956842be8a=1600527516; srcid=eyJ0Ijoie1wiZGF0YVwiOlwiYTE5NzE1ZDYzZjIzMDQ0NTQ5NWUzYmFlOTQxYTI1Nzg0YWZlZjk5MDI4NDJjYjU5YWE1YjEyYTg5YzYzODM5ODU3YmJlNTc1NGU3OTlkODU3NzA1NGM2ZGRkY2YxNWYxMjdiMGYyNGQ3ZjEzNTdmOWM1NDRmMjgxYjRhM2NlYjU2MDBlZGUxZDA0YjgxMWFmZTRhODk2ZGM3YjRlMmEwMWVjNDQ4YjQ1NjJmODQ2MWM2MDRiM2Q1N2NlMjk1YmZkYzk0YTFjMjUzYzkyYTM2NDBjOWUzNzg3ZjkyOWI2MmE2MTRlMzVhNmU4NDJmMDlkNWNhNGM4MDM0Y2ZhNzMzYjBlZTViYzg4OTA2YWNmNWNjM2Y3YmQ1MjQwMTA4MDVmMGY4ZTE0OTAzNjNjMjVmZDcwN2RhNWU3YjIzMTE2ZTliM2QyYjM5ZTJmYjQ3NmJkMWQ5MTA0NDA4ZTMzMzJmZVwiLFwia2V5X2lkXCI6XCIxXCIsXCJzaWduXCI6XCJkZmYxZTM4NVwifSIsInIiOiJodHRwczovL3htLmxpYW5qaWEuY29tL2Vyc2hvdWZhbmcvIiwib3MiOiJ3ZWIiLCJ2IjoiMC4xIn0='

    # Cookie转化为字典
    cookies = cookies.split('; ')
    cookies_dict = {}
    for i in cookies:
        cookies_dict[i.split('=')[0]] = i.split('=')[1]

    return user_agent, cookies_dict

## 获取每个房源的链接

In [36]:
def get_urls(url, u_a, c_d):
    html = requests.get(url, headers=u_a, cookies=c_d)
    soup = BeautifulSoup(html.text, 'lxml')  # html.parser解析失败
    items = soup.find('div', class_='content').find('div', class_='leftContent').find(
        'ul', class_='listContent').find_all('li')
    hrefs = []
    for item in items:
        info = {}
        info['页面链接']=url
        info['房源链接'] = item.find('div', class_='info').find(
            'div', class_='title').find('a')['href']
        info['小区名称'] = item.find('div', class_='info').find(
            'div', class_='title').find('a').text
        info['成交时间'] = item.find('div', class_='info').find(
            'div', class_='address').find('div', class_='dealDate').text
        hrefs.append(info)
    return hrefs

# get_urls('https://xm.lianjia.com/chengjiao/pg2/', u_a, c_d)

## 获取每个房源详细信息

In [5]:

def get_info(url, u_a, c_d):
    html = requests.get(url, headers=u_a, cookies=c_d)
    html.encoding = html.apparent_encoding  # 解决乱码的万金油方法
    soup = BeautifulSoup(html.text, 'html.parser')

    results = []
    info = {}

    a = soup.find('div', class_='overview').find('div', class_='info fr')
    info['房源链接'] = url
    info['总价'] = a.find('div', class_='price').find(
        'span', class_='dealTotalPrice').text
    info['单价'] = a.find('div', class_='price').text.split('万')[1]
    info['挂牌价格'] = a.find('div', class_='msg').find_all('span')[0].text
    info['成交周期'] = a.find('div', class_='msg').find_all('span')[1].text
#     info['小区名称'] = a.find('div', class_='aroundInfo').find(
#         'div', class_='communityName').find('a').text
#     info['所在大区'] = a.find('div', class_='aroundInfo').find(
#         'div', class_='areaName').find('span', class_='info').find_all('a')[0].text
#     info['所在详细区域'] = a.find('div', class_='aroundInfo').find(
#         'div', class_='areaName').find('span', class_='info').find_all('a')[1].text

    b = soup.find('div', class_='newwrap baseinform').find('div', class_='introContent')

    # 基本属性
    items1 = b.find('div', class_='content').find_all('li')
    for item in items1:
        info[item.text.strip()[:4]] = item.text.strip()[4:]

    # 交易属性
    items2 = b.find('div', class_='transaction').find('div',class_='content').find_all('li')
    for item in items2:
        info[item.text.strip()[:4]] = item.text.strip()[4:]

    results.append(info)
    return results


# get_info('https://xm.lianjia.com/chengjiao/105105116517.html', u_a, c_d)

## 设置主函数

In [6]:
# def main():
#     house_urls = []  # 保存房源链接

#     print('开始采集房源链接！预计耗时1小时......')

#     house_urls_error = []

#     login = ua_ck()
#     u_a = login[0]
#     c_d = login[1]

#     for url in house_urls:
#         try:
#             href = get_urls(url, u_a, c_d)
#             house_urls.extend(href)
#         except:
#             house_urls_error.append(url)
#         time.sleep(random.random()*3)

#     print('房源链接采集完成！共采集{}条链接，开始采集房源信息，预计耗时10小时......'.format(len(house_urls)))

#     time.sleep(5)

#     data = []  # 保存房源详细信息
#     data_error = []
#     for house_url in house_urls:
#         try:
#             info = get_info(house_url, u_a, c_d)
#             data.extend(info)
#         except:
#             data_error.append(house_url)
#         time.sleep(random.random()*3)
#         print('已采集{}条数据'.format(len(data)))  # 监控采集进度

#     print('房源信息采集完成！！！总共采集了{}条数据'.format(len(data)))

#     return data, house_urls, house_urls_error, data_error

直接用主函数采集房源链接和房源信息，容易失败，建议分批采集

## 采集房源链接

In [6]:
login = ua_ck()
u_a = login[0]
c_d = login[1]

In [37]:
house_urls = []  # 保存房源链接
error_url = []  # 保存采集失败的页面链接
for url in xm_urls_list:
    try:
        href = get_urls(url, u_a, c_d)
        house_urls.extend(href)
    except:
        error_url.append(url)
    time.sleep(random.random()*3)  # 设置时间间隔
    
    print('已采集{}条数据'.format(len(house_urls)))  # 监控采集进度

print('房源链接采集完成！！！总共采集了{}条链接'.format(len(house_urls)))

已采集30条数据
已采集60条数据
已采集90条数据
已采集120条数据
已采集150条数据
已采集180条数据
已采集210条数据
已采集240条数据
已采集270条数据
已采集300条数据
已采集330条数据
已采集360条数据
已采集390条数据
已采集420条数据
已采集450条数据
已采集480条数据
已采集510条数据
已采集540条数据
已采集570条数据
已采集600条数据
已采集630条数据
已采集660条数据
已采集690条数据
已采集720条数据
已采集750条数据
已采集780条数据
已采集810条数据
已采集840条数据
已采集870条数据
已采集900条数据
已采集930条数据
已采集960条数据
已采集990条数据
已采集1020条数据
已采集1050条数据
已采集1080条数据
已采集1110条数据
已采集1139条数据
已采集1169条数据
已采集1199条数据
已采集1229条数据
已采集1259条数据
已采集1289条数据
已采集1319条数据
已采集1349条数据
已采集1379条数据
已采集1409条数据
已采集1439条数据
已采集1469条数据
已采集1499条数据
已采集1529条数据
已采集1559条数据
已采集1589条数据
已采集1619条数据
已采集1649条数据
已采集1679条数据
已采集1709条数据
已采集1739条数据
已采集1769条数据
已采集1799条数据
已采集1829条数据
已采集1859条数据
已采集1889条数据
已采集1919条数据
已采集1949条数据
已采集1979条数据
已采集2009条数据
已采集2039条数据
已采集2069条数据
已采集2099条数据
已采集2129条数据
已采集2159条数据
已采集2189条数据
已采集2219条数据
已采集2249条数据
已采集2279条数据
已采集2309条数据
已采集2339条数据
已采集2369条数据
已采集2399条数据
已采集2429条数据
已采集2459条数据
已采集2489条数据
已采集2519条数据
已采集2549条数据
已采集2579条数据
已采集2609条数据
已采集2639条数据
已采集2669条数据
已采集2699条数据
已采集2729条数据
已采集2759条数据
已采集2789条数据
已采集2819条数据
已采

In [30]:
house_urls[:5]

[{'房源链接': 'https://xm.lianjia.com/chengjiao/105104758139.html',
  '小区名称': '金鸡亭小区西区 3室1厅 75.83平米',
  '成交时间': '2020.09.09'},
 {'房源链接': 'https://xm.lianjia.com/chengjiao/105104759843.html',
  '小区名称': '龙山山庄 3室2厅 113.89平米',
  '成交时间': '2020.09.09'},
 {'房源链接': 'https://xm.lianjia.com/chengjiao/105105280722.html',
  '小区名称': '前埔北区一里 3室2厅 108.58平米',
  '成交时间': '2020.09.09'},
 {'房源链接': 'https://xm.lianjia.com/chengjiao/105102055905.html',
  '小区名称': '湖东花园 4室2厅 164.65平米',
  '成交时间': '2020.09.09'},
 {'房源链接': 'https://xm.lianjia.com/chengjiao/105102205483.html',
  '小区名称': '盈翠里 3室1厅 89.89平米',
  '成交时间': '2020.09.09'},
 {'房源链接': 'https://xm.lianjia.com/chengjiao/105104428036.html',
  '小区名称': '禾祥首府 1室0厅 48.98平米',
  '成交时间': '2020.09.09'},
 {'房源链接': 'https://xm.lianjia.com/chengjiao/105102389522.html',
  '小区名称': '国贸汇景 4室2厅 154.59平米',
  '成交时间': '2020.09.09'},
 {'房源链接': 'https://xm.lianjia.com/chengjiao/105104970100.html',
  '小区名称': '万寿路 2室2厅 65.42平米',
  '成交时间': '2020.09.09'},
 {'房源链接': 'https://xm.lianjia.com

In [27]:
error_url  # 查看采集失败的页面链接

[]

In [38]:
# 把链接先保存起来
save_list = pd.DataFrame(house_urls)
save_list.to_excel(r'厦门成交的二手房源链接_{}.xlsx'.format(date.today()))

In [11]:
house_urls_list=[]
for i in house_urls:
    house_urls_list.append(i['房源链接'])
house_urls_list[:5]

['https://xm.lianjia.com/chengjiao/105104758139.html',
 'https://xm.lianjia.com/chengjiao/105104759843.html',
 'https://xm.lianjia.com/chengjiao/105105280722.html',
 'https://xm.lianjia.com/chengjiao/105102055905.html',
 'https://xm.lianjia.com/chengjiao/105102205483.html',
 'https://xm.lianjia.com/chengjiao/105104428036.html',
 'https://xm.lianjia.com/chengjiao/105102389522.html',
 'https://xm.lianjia.com/chengjiao/105104970100.html',
 'https://xm.lianjia.com/chengjiao/105103261724.html',
 'https://xm.lianjia.com/chengjiao/105104754253.html',
 'https://xm.lianjia.com/chengjiao/105105116517.html',
 'https://xm.lianjia.com/chengjiao/105103154751.html',
 'https://xm.lianjia.com/chengjiao/105105251523.html',
 'https://xm.lianjia.com/chengjiao/105105010437.html',
 'https://xm.lianjia.com/chengjiao/105105216462.html',
 'https://xm.lianjia.com/chengjiao/105104866060.html',
 'https://xm.lianjia.com/chengjiao/105103398729.html',
 'https://xm.lianjia.com/chengjiao/105104809602.html',
 'https://

## 分批采集房源信息

In [12]:
data = []  # 保存房源详细信息
error_data = []  # 保存采集失败的链接

In [13]:
# 采集前3000条
for house_url in house_urls_list[:3000]:
    try:
        info = get_info(house_url, u_a, c_d)
        data.extend(info)
    except:
        error_data.append(house_url)
        print('采集失败{}条'.format(len(error_data)))
    time.sleep(random.random()*3)  # 设置时间间隔
    
    print('已采集{}条数据'.format(len(data)))  # 监控采集进度

print('房源信息采集完成！！！总共采集了{}条数据'.format(len(data)))

已采集1条数据
已采集2条数据
已采集3条数据
已采集4条数据
已采集5条数据
已采集6条数据
已采集7条数据
已采集8条数据
已采集9条数据
已采集10条数据
已采集11条数据
已采集12条数据
已采集13条数据
已采集14条数据
已采集15条数据
已采集16条数据
已采集17条数据
已采集18条数据
已采集19条数据
已采集20条数据
已采集21条数据
已采集22条数据
已采集23条数据
已采集24条数据
已采集25条数据
已采集26条数据
已采集27条数据
已采集28条数据
已采集29条数据
已采集30条数据
已采集31条数据
已采集32条数据
已采集33条数据
已采集34条数据
已采集35条数据
已采集36条数据
已采集37条数据
已采集38条数据
已采集39条数据
已采集40条数据
已采集41条数据
已采集42条数据
已采集43条数据
已采集44条数据
已采集45条数据
已采集46条数据
已采集47条数据
已采集48条数据
已采集49条数据
已采集50条数据
已采集51条数据
已采集52条数据
已采集53条数据
已采集54条数据
已采集55条数据
已采集56条数据
已采集57条数据
已采集58条数据
已采集59条数据
已采集60条数据
已采集61条数据
已采集62条数据
已采集63条数据
已采集64条数据
已采集65条数据
已采集66条数据
已采集67条数据
已采集68条数据
已采集69条数据
已采集70条数据
已采集71条数据
已采集72条数据
已采集73条数据
已采集74条数据
已采集75条数据
已采集76条数据
已采集77条数据
已采集78条数据
已采集79条数据
已采集80条数据
已采集81条数据
已采集82条数据
已采集83条数据
已采集84条数据
已采集85条数据
已采集86条数据
已采集87条数据
已采集88条数据
已采集89条数据
已采集90条数据
已采集91条数据
已采集92条数据
已采集93条数据
已采集94条数据
已采集95条数据
已采集96条数据
已采集97条数据
已采集98条数据
已采集99条数据
已采集100条数据
已采集101条数据
已采集102条数据
已采集103条数据
已采集104条数据
已采集105条数据
已采集106条数据
已采集107条数据
已采集108条数据
已采集109条数据
已采集110条数据
已采集111条数

已采集590条数据
已采集591条数据
已采集592条数据
已采集593条数据
已采集594条数据
已采集595条数据
已采集596条数据
已采集597条数据
已采集598条数据
已采集599条数据
已采集600条数据
已采集601条数据
已采集602条数据
已采集603条数据
已采集604条数据
已采集605条数据
已采集606条数据
已采集607条数据
已采集608条数据
已采集609条数据
已采集610条数据
已采集611条数据
已采集612条数据
已采集613条数据
已采集614条数据
已采集615条数据
已采集616条数据
已采集617条数据
已采集618条数据
已采集619条数据
已采集620条数据
已采集621条数据
已采集622条数据
已采集623条数据
已采集624条数据
已采集625条数据
已采集626条数据
已采集627条数据
已采集628条数据
已采集629条数据
已采集630条数据
已采集631条数据
已采集632条数据
已采集633条数据
已采集634条数据
已采集635条数据
已采集636条数据
已采集637条数据
已采集638条数据
已采集639条数据
已采集640条数据
已采集641条数据
已采集642条数据
已采集643条数据
已采集644条数据
已采集645条数据
已采集646条数据
已采集647条数据
已采集648条数据
已采集649条数据
已采集650条数据
已采集651条数据
采集失败134条
已采集651条数据
已采集652条数据
已采集653条数据
已采集654条数据
已采集655条数据
已采集656条数据
已采集657条数据
已采集658条数据
已采集659条数据
已采集660条数据
已采集661条数据
已采集662条数据
已采集663条数据
已采集664条数据
已采集665条数据
已采集666条数据
已采集667条数据
已采集668条数据
已采集669条数据
已采集670条数据
已采集671条数据
已采集672条数据
已采集673条数据
已采集674条数据
已采集675条数据
已采集676条数据
已采集677条数据
已采集678条数据
已采集679条数据
已采集680条数据
已采集681条数据
已采集682条数据
已采集683条数据
已采集684条数据
已采集685条数据
已采集686条数据
已采集687条数据
已

已采集1342条数据
采集失败151条
已采集1342条数据
采集失败152条
已采集1342条数据
采集失败153条
已采集1342条数据
采集失败154条
已采集1342条数据
采集失败155条
已采集1342条数据
采集失败156条
已采集1342条数据
采集失败157条
已采集1342条数据
采集失败158条
已采集1342条数据
采集失败159条
已采集1342条数据
采集失败160条
已采集1342条数据
采集失败161条
已采集1342条数据
采集失败162条
已采集1342条数据
采集失败163条
已采集1342条数据
采集失败164条
已采集1342条数据
采集失败165条
已采集1342条数据
采集失败166条
已采集1342条数据
采集失败167条
已采集1342条数据
采集失败168条
已采集1342条数据
采集失败169条
已采集1342条数据
采集失败170条
已采集1342条数据
采集失败171条
已采集1342条数据
采集失败172条
已采集1342条数据
采集失败173条
已采集1342条数据
采集失败174条
已采集1342条数据
采集失败175条
已采集1342条数据
采集失败176条
已采集1342条数据
采集失败177条
已采集1342条数据
采集失败178条
已采集1342条数据
采集失败179条
已采集1342条数据
采集失败180条
已采集1342条数据
采集失败181条
已采集1342条数据
采集失败182条
已采集1342条数据
采集失败183条
已采集1342条数据
采集失败184条
已采集1342条数据
采集失败185条
已采集1342条数据
采集失败186条
已采集1342条数据
采集失败187条
已采集1342条数据
采集失败188条
已采集1342条数据
采集失败189条
已采集1342条数据
采集失败190条
已采集1342条数据
采集失败191条
已采集1342条数据
采集失败192条
已采集1342条数据
采集失败193条
已采集1342条数据
采集失败194条
已采集1342条数据
采集失败195条
已采集1342条数据
采集失败196条
已采集1342条数据
采集失败197条
已采集1342条数据
采集失败198条
已采集1342条数据
采集失败199条
已采集1342条数据
采集失败200条


已采集1540条数据
已采集1541条数据
已采集1542条数据
已采集1543条数据
已采集1544条数据
已采集1545条数据
已采集1546条数据
已采集1547条数据
已采集1548条数据
已采集1549条数据
已采集1550条数据
已采集1551条数据
已采集1552条数据
已采集1553条数据
已采集1554条数据
已采集1555条数据
已采集1556条数据
已采集1557条数据
已采集1558条数据
已采集1559条数据
已采集1560条数据
已采集1561条数据
已采集1562条数据
已采集1563条数据
已采集1564条数据
已采集1565条数据
已采集1566条数据
已采集1567条数据
已采集1568条数据
已采集1569条数据
已采集1570条数据
已采集1571条数据
已采集1572条数据
已采集1573条数据
已采集1574条数据
已采集1575条数据
已采集1576条数据
已采集1577条数据
已采集1578条数据
已采集1579条数据
已采集1580条数据
已采集1581条数据
已采集1582条数据
已采集1583条数据
已采集1584条数据
已采集1585条数据
已采集1586条数据
已采集1587条数据
已采集1588条数据
已采集1589条数据
已采集1590条数据
已采集1591条数据
已采集1592条数据
已采集1593条数据
已采集1594条数据
已采集1595条数据
已采集1596条数据
已采集1597条数据
已采集1598条数据
已采集1599条数据
已采集1600条数据
已采集1601条数据
已采集1602条数据
已采集1603条数据
已采集1604条数据
已采集1605条数据
已采集1606条数据
已采集1607条数据
已采集1608条数据
已采集1609条数据
已采集1610条数据
已采集1611条数据
已采集1612条数据
已采集1613条数据
已采集1614条数据
已采集1615条数据
已采集1616条数据
已采集1617条数据
已采集1618条数据
已采集1619条数据
已采集1620条数据
已采集1621条数据
已采集1622条数据
已采集1623条数据
已采集1624条数据
已采集1625条数据
已采集1626条数据
已采集1627条数据
已采集1628条数据
已采集1629条数据
已采集1630条数据

已采集2285条数据
已采集2286条数据
已采集2287条数据
已采集2288条数据
已采集2289条数据
已采集2290条数据
已采集2291条数据
已采集2292条数据
已采集2293条数据
已采集2294条数据
已采集2295条数据
已采集2296条数据
已采集2297条数据
已采集2298条数据
已采集2299条数据
已采集2300条数据
已采集2301条数据
已采集2302条数据
已采集2303条数据
已采集2304条数据
已采集2305条数据
已采集2306条数据
已采集2307条数据
已采集2308条数据
已采集2309条数据
已采集2310条数据
已采集2311条数据
已采集2312条数据
已采集2313条数据
已采集2314条数据
已采集2315条数据
已采集2316条数据
已采集2317条数据
已采集2318条数据
已采集2319条数据
已采集2320条数据
已采集2321条数据
已采集2322条数据
已采集2323条数据
已采集2324条数据
已采集2325条数据
已采集2326条数据
已采集2327条数据
已采集2328条数据
已采集2329条数据
已采集2330条数据
已采集2331条数据
已采集2332条数据
已采集2333条数据
已采集2334条数据
已采集2335条数据
已采集2336条数据
已采集2337条数据
已采集2338条数据
已采集2339条数据
已采集2340条数据
已采集2341条数据
已采集2342条数据
已采集2343条数据
已采集2344条数据
已采集2345条数据
已采集2346条数据
已采集2347条数据
已采集2348条数据
已采集2349条数据
已采集2350条数据
已采集2351条数据
已采集2352条数据
已采集2353条数据
已采集2354条数据
已采集2355条数据
已采集2356条数据
已采集2357条数据
已采集2358条数据
已采集2359条数据
已采集2360条数据
已采集2361条数据
已采集2362条数据
已采集2363条数据
已采集2364条数据
已采集2365条数据
已采集2366条数据
已采集2367条数据
已采集2368条数据
已采集2369条数据
已采集2370条数据
已采集2371条数据
已采集2372条数据
已采集2373条数据
已采集2374条数据
已采集2375条数据

In [14]:
error_data

[]

In [16]:
data[:10]

[{'房源链接': 'https://xm.lianjia.com/chengjiao/105104758139.html',
  '总价': '300万',
  '单价': '39563元/平',
  '挂牌价格': '316挂牌价格（万）',
  '成交周期': '67成交周期（天）',
  '房屋户型': '3室1厅1厨1卫',
  '所在楼层': '中楼层(共7层)',
  '建筑面积': '75.83㎡',
  '户型结构': '平层',
  '套内面积': '暂无数据',
  '建筑类型': '板楼',
  '房屋朝向': '南',
  '建成年代': '1997',
  '装修情况': '精装',
  '建筑结构': '钢混结构',
  '供暖方式': '',
  '梯户比例': '一梯四户',
  '配备电梯': '无',
  '链家编号': '105104758139',
  '交易权属': '商品房',
  '挂牌时间': '2020-07-05',
  '房屋用途': '普通住宅',
  '房屋年限': '满五年',
  '房权所属': '非共有'},
 {'房源链接': 'https://xm.lianjia.com/chengjiao/105104759843.html',
  '总价': '456万',
  '单价': '40039元/平',
  '挂牌价格': '469.8挂牌价格（万）',
  '成交周期': '67成交周期（天）',
  '房屋户型': '3室2厅1厨2卫',
  '所在楼层': '中楼层(共8层)',
  '建筑面积': '113.89㎡',
  '户型结构': '平层',
  '套内面积': '暂无数据',
  '建筑类型': '板楼',
  '房屋朝向': '南',
  '建成年代': '1996',
  '装修情况': '精装',
  '建筑结构': '钢混结构',
  '供暖方式': '',
  '梯户比例': '一梯两户',
  '配备电梯': '无',
  '链家编号': '105104759843',
  '交易权属': '商品房',
  '挂牌时间': '2020-07-05',
  '房屋用途': '普通住宅',
  '房屋年限': '暂无数据',
  '房权所属': '非共有'},
 {'房源链

In [13]:
# 采集前3000条
for house_url in house_urls_list[3000:]:
    try:
        info = get_info(house_url, u_a, c_d)
        data.extend(info)
    except:
        error_data.append(house_url)
        print('采集失败{}条'.format(len(error_data)))
    time.sleep(random.random()*3)  # 设置时间间隔
    
    print('已采集{}条数据'.format(len(data)))  # 监控采集进度

print('房源信息采集完成！！！总共采集了{}条数据'.format(len(data)))

已采集1条数据
已采集2条数据
已采集3条数据
已采集4条数据
已采集5条数据
已采集6条数据
已采集7条数据
已采集8条数据
已采集9条数据
已采集10条数据
已采集11条数据
已采集12条数据
已采集13条数据
已采集14条数据
已采集15条数据
已采集16条数据
已采集17条数据
已采集18条数据
已采集19条数据
已采集20条数据
已采集21条数据
已采集22条数据
已采集23条数据
已采集24条数据
已采集25条数据
已采集26条数据
已采集27条数据
已采集28条数据
已采集29条数据
已采集30条数据
已采集31条数据
已采集32条数据
已采集33条数据
已采集34条数据
已采集35条数据
已采集36条数据
已采集37条数据
已采集38条数据
已采集39条数据
已采集40条数据
已采集41条数据
已采集42条数据
已采集43条数据
已采集44条数据
已采集45条数据
已采集46条数据
已采集47条数据
已采集48条数据
已采集49条数据
已采集50条数据
已采集51条数据
已采集52条数据
已采集53条数据
已采集54条数据
已采集55条数据
已采集56条数据
已采集57条数据
已采集58条数据
已采集59条数据
已采集60条数据
已采集61条数据
已采集62条数据
已采集63条数据
已采集64条数据
已采集65条数据
已采集66条数据
已采集67条数据
已采集68条数据
已采集69条数据
已采集70条数据
已采集71条数据
已采集72条数据
已采集73条数据
已采集74条数据
已采集75条数据
已采集76条数据
已采集77条数据
已采集78条数据
已采集79条数据
已采集80条数据
已采集81条数据
已采集82条数据
已采集83条数据
已采集84条数据
已采集85条数据
已采集86条数据
已采集87条数据
已采集88条数据
已采集89条数据
已采集90条数据
已采集91条数据
已采集92条数据
已采集93条数据
已采集94条数据
已采集95条数据
已采集96条数据
已采集97条数据
已采集98条数据
已采集99条数据
已采集100条数据
已采集101条数据
已采集102条数据
已采集103条数据
已采集104条数据
已采集105条数据
已采集106条数据
已采集107条数据
已采集108条数据
已采集109条数据
已采集110条数据
已采集111条数

已采集832条数据
已采集833条数据
已采集834条数据
已采集835条数据
已采集836条数据
已采集837条数据
已采集838条数据
已采集839条数据
已采集840条数据
已采集841条数据
已采集842条数据
已采集843条数据
已采集844条数据
已采集845条数据
已采集846条数据
已采集847条数据
已采集848条数据
已采集849条数据
已采集850条数据
已采集851条数据
已采集852条数据
已采集853条数据
已采集854条数据
已采集855条数据
已采集856条数据
已采集857条数据
已采集858条数据
已采集859条数据
已采集860条数据
已采集861条数据
已采集862条数据
已采集863条数据
已采集864条数据
已采集865条数据
已采集866条数据
已采集867条数据
已采集868条数据
已采集869条数据
已采集870条数据
已采集871条数据
已采集872条数据
已采集873条数据
已采集874条数据
已采集875条数据
已采集876条数据
已采集877条数据
已采集878条数据
已采集879条数据
已采集880条数据
已采集881条数据
已采集882条数据
已采集883条数据
已采集884条数据
已采集885条数据
已采集886条数据
已采集887条数据
已采集888条数据
已采集889条数据
已采集890条数据
已采集891条数据
已采集892条数据
已采集893条数据
已采集894条数据
已采集895条数据
已采集896条数据
已采集897条数据
已采集898条数据
已采集899条数据
已采集900条数据
已采集901条数据
已采集902条数据
已采集903条数据
已采集904条数据
已采集905条数据
已采集906条数据
已采集907条数据
已采集908条数据
已采集909条数据
已采集910条数据
已采集911条数据
已采集912条数据
已采集913条数据
已采集914条数据
已采集915条数据
已采集916条数据
已采集917条数据
已采集918条数据
已采集919条数据
已采集920条数据
已采集921条数据
已采集922条数据
已采集923条数据
已采集924条数据
已采集925条数据
已采集926条数据
已采集927条数据
已采集928条数据
已采集929条数据
已采集930条数据
已采集931条数据


已采集1593条数据
已采集1594条数据
已采集1595条数据
已采集1596条数据
已采集1597条数据
已采集1598条数据
已采集1599条数据
已采集1600条数据
已采集1601条数据
已采集1602条数据
已采集1603条数据
已采集1604条数据
已采集1605条数据
已采集1606条数据
已采集1607条数据
已采集1608条数据
已采集1609条数据
已采集1610条数据
已采集1611条数据
已采集1612条数据
已采集1613条数据
已采集1614条数据
已采集1615条数据
已采集1616条数据
已采集1617条数据
已采集1618条数据
已采集1619条数据
已采集1620条数据
已采集1621条数据
已采集1622条数据
已采集1623条数据
已采集1624条数据
已采集1625条数据
已采集1626条数据
已采集1627条数据
已采集1628条数据
已采集1629条数据
已采集1630条数据
已采集1631条数据
已采集1632条数据
已采集1633条数据
已采集1634条数据
已采集1635条数据
已采集1636条数据
已采集1637条数据
已采集1638条数据
已采集1639条数据
已采集1640条数据
已采集1641条数据
已采集1642条数据
已采集1643条数据
已采集1644条数据
已采集1645条数据
已采集1646条数据
已采集1647条数据
已采集1648条数据
已采集1649条数据
已采集1650条数据
已采集1651条数据
已采集1652条数据
已采集1653条数据
已采集1654条数据
已采集1655条数据
已采集1656条数据
已采集1657条数据
已采集1658条数据
已采集1659条数据
已采集1660条数据
已采集1661条数据
已采集1662条数据
已采集1663条数据
已采集1664条数据
已采集1665条数据
已采集1666条数据
已采集1667条数据
已采集1668条数据
已采集1669条数据
已采集1670条数据
已采集1671条数据
已采集1672条数据
已采集1673条数据
已采集1674条数据
已采集1675条数据
已采集1676条数据
已采集1677条数据
已采集1678条数据
已采集1679条数据
已采集1680条数据
已采集1681条数据
已采集1682条数据
已采集1683条数据

已采集2338条数据
已采集2339条数据
已采集2340条数据
已采集2341条数据
已采集2342条数据
已采集2343条数据
已采集2344条数据
已采集2345条数据
已采集2346条数据
已采集2347条数据
已采集2348条数据
已采集2349条数据
已采集2350条数据
已采集2351条数据
已采集2352条数据
已采集2353条数据
已采集2354条数据
已采集2355条数据
已采集2356条数据
已采集2357条数据
已采集2358条数据
已采集2359条数据
已采集2360条数据
已采集2361条数据
已采集2362条数据
已采集2363条数据
已采集2364条数据
已采集2365条数据
已采集2366条数据
已采集2367条数据
已采集2368条数据
已采集2369条数据
已采集2370条数据
已采集2371条数据
已采集2372条数据
已采集2373条数据
已采集2374条数据
已采集2375条数据
已采集2376条数据
已采集2377条数据
已采集2378条数据
已采集2379条数据
已采集2380条数据
已采集2381条数据
已采集2382条数据
已采集2383条数据
已采集2384条数据
已采集2385条数据
已采集2386条数据
已采集2387条数据
已采集2388条数据
已采集2389条数据
已采集2390条数据
已采集2391条数据
已采集2392条数据
已采集2393条数据
已采集2394条数据
已采集2395条数据
已采集2396条数据
已采集2397条数据
已采集2398条数据
已采集2399条数据
已采集2400条数据
已采集2401条数据
已采集2402条数据
已采集2403条数据
已采集2404条数据
已采集2405条数据
已采集2406条数据
已采集2407条数据
已采集2408条数据
已采集2409条数据
已采集2410条数据
已采集2411条数据
已采集2412条数据
已采集2413条数据
已采集2414条数据
已采集2415条数据
已采集2416条数据
已采集2417条数据
已采集2418条数据
已采集2419条数据
已采集2420条数据
已采集2421条数据
已采集2422条数据
已采集2423条数据
已采集2424条数据
已采集2425条数据
已采集2426条数据
已采集2427条数据
已采集2428条数据

已采集3075条数据
已采集3076条数据
已采集3077条数据
已采集3078条数据
已采集3079条数据
已采集3080条数据
已采集3081条数据
已采集3082条数据
已采集3083条数据
已采集3084条数据
已采集3085条数据
已采集3086条数据
已采集3087条数据
已采集3088条数据
已采集3089条数据
已采集3090条数据
已采集3091条数据
已采集3092条数据
已采集3093条数据
已采集3094条数据
已采集3095条数据
已采集3096条数据
已采集3097条数据
已采集3098条数据
已采集3099条数据
已采集3100条数据
已采集3101条数据
已采集3102条数据
已采集3103条数据
已采集3104条数据
已采集3105条数据
已采集3106条数据
已采集3107条数据
已采集3108条数据
已采集3109条数据
已采集3110条数据
已采集3111条数据
已采集3112条数据
已采集3113条数据
已采集3114条数据
已采集3115条数据
已采集3116条数据
已采集3117条数据
已采集3118条数据
已采集3119条数据
已采集3120条数据
已采集3121条数据
已采集3122条数据
已采集3123条数据
已采集3124条数据
已采集3125条数据
已采集3126条数据
已采集3127条数据
已采集3128条数据
已采集3129条数据
已采集3130条数据
已采集3131条数据
已采集3132条数据
已采集3133条数据
已采集3134条数据
已采集3135条数据
已采集3136条数据
已采集3137条数据
已采集3138条数据
已采集3139条数据
已采集3140条数据
已采集3141条数据
已采集3142条数据
已采集3143条数据
已采集3144条数据
已采集3145条数据
已采集3146条数据
已采集3147条数据
已采集3148条数据
已采集3149条数据
已采集3150条数据
已采集3151条数据
已采集3152条数据
已采集3153条数据
已采集3154条数据
已采集3155条数据
已采集3156条数据
已采集3157条数据
已采集3158条数据
已采集3159条数据
已采集3160条数据
已采集3161条数据
已采集3162条数据
已采集3163条数据
已采集3164条数据
已采集3165条数据

已采集3820条数据
已采集3821条数据
已采集3822条数据
已采集3823条数据
已采集3824条数据
已采集3825条数据
已采集3826条数据
已采集3827条数据
已采集3828条数据
已采集3829条数据
已采集3830条数据
已采集3831条数据
已采集3832条数据
已采集3833条数据
已采集3834条数据
已采集3835条数据
已采集3836条数据
已采集3837条数据
已采集3838条数据
已采集3839条数据
已采集3840条数据
已采集3841条数据
已采集3842条数据
已采集3843条数据
已采集3844条数据
已采集3845条数据
已采集3846条数据
已采集3847条数据
已采集3848条数据
已采集3849条数据
已采集3850条数据
已采集3851条数据
已采集3852条数据
已采集3853条数据
已采集3854条数据
已采集3855条数据
已采集3856条数据
已采集3857条数据
已采集3858条数据
已采集3859条数据
已采集3860条数据
已采集3861条数据
已采集3862条数据
已采集3863条数据
已采集3864条数据
已采集3865条数据
已采集3866条数据
已采集3867条数据
已采集3868条数据
已采集3869条数据
已采集3870条数据
已采集3871条数据
已采集3872条数据
已采集3873条数据
已采集3874条数据
已采集3875条数据
已采集3876条数据
已采集3877条数据
已采集3878条数据
已采集3879条数据
已采集3880条数据
已采集3881条数据
已采集3882条数据
已采集3883条数据
已采集3884条数据
已采集3885条数据
已采集3886条数据
已采集3887条数据
已采集3888条数据
已采集3889条数据
已采集3890条数据
已采集3891条数据
已采集3892条数据
已采集3893条数据
已采集3894条数据
已采集3895条数据
已采集3896条数据
已采集3897条数据
已采集3898条数据
已采集3899条数据
已采集3900条数据
已采集3901条数据
已采集3902条数据
已采集3903条数据
已采集3904条数据
已采集3905条数据
已采集3906条数据
已采集3907条数据
已采集3908条数据
已采集3909条数据
已采集3910条数据

已采集4565条数据
已采集4566条数据
已采集4567条数据
已采集4568条数据
已采集4569条数据
已采集4570条数据
已采集4571条数据
已采集4572条数据
已采集4573条数据
已采集4574条数据
已采集4575条数据
已采集4576条数据
已采集4577条数据
已采集4578条数据
已采集4579条数据
已采集4580条数据
已采集4581条数据
已采集4582条数据
已采集4583条数据
已采集4584条数据
已采集4585条数据
已采集4586条数据
已采集4587条数据
已采集4588条数据
已采集4589条数据
已采集4590条数据
已采集4591条数据
已采集4592条数据
已采集4593条数据
已采集4594条数据
已采集4595条数据
已采集4596条数据
已采集4597条数据
已采集4598条数据
已采集4599条数据
已采集4600条数据
已采集4601条数据
已采集4602条数据
已采集4603条数据
已采集4604条数据
已采集4605条数据
已采集4606条数据
已采集4607条数据
已采集4608条数据
已采集4609条数据
已采集4610条数据
已采集4611条数据
已采集4612条数据
已采集4613条数据
已采集4614条数据
已采集4615条数据
已采集4616条数据
已采集4617条数据
已采集4618条数据
已采集4619条数据
已采集4620条数据
已采集4621条数据
已采集4622条数据
已采集4623条数据
已采集4624条数据
已采集4625条数据
已采集4626条数据
已采集4627条数据
已采集4628条数据
已采集4629条数据
已采集4630条数据
已采集4631条数据
已采集4632条数据
已采集4633条数据
已采集4634条数据
已采集4635条数据
已采集4636条数据
已采集4637条数据
已采集4638条数据
已采集4639条数据
已采集4640条数据
已采集4641条数据
已采集4642条数据
已采集4643条数据
已采集4644条数据
已采集4645条数据
已采集4646条数据
已采集4647条数据
已采集4648条数据
已采集4649条数据
已采集4650条数据
已采集4651条数据
已采集4652条数据
已采集4653条数据
已采集4654条数据
已采集4655条数据

已采集5310条数据
已采集5311条数据
已采集5312条数据
已采集5313条数据
已采集5314条数据
已采集5315条数据
已采集5316条数据
已采集5317条数据
已采集5318条数据
已采集5319条数据
已采集5320条数据
已采集5321条数据
已采集5322条数据
已采集5323条数据
已采集5324条数据
已采集5325条数据
已采集5326条数据
已采集5327条数据
已采集5328条数据
已采集5329条数据
已采集5330条数据
已采集5331条数据
已采集5332条数据
已采集5333条数据
已采集5334条数据
已采集5335条数据
已采集5336条数据
已采集5337条数据
已采集5338条数据
已采集5339条数据
已采集5340条数据
已采集5341条数据
已采集5342条数据
已采集5343条数据
已采集5344条数据
已采集5345条数据
已采集5346条数据
已采集5347条数据
已采集5348条数据
已采集5349条数据
已采集5350条数据
已采集5351条数据
已采集5352条数据
已采集5353条数据
已采集5354条数据
已采集5355条数据
已采集5356条数据
已采集5357条数据
已采集5358条数据
已采集5359条数据
已采集5360条数据
已采集5361条数据
已采集5362条数据
已采集5363条数据
已采集5364条数据
已采集5365条数据
已采集5366条数据
已采集5367条数据
已采集5368条数据
已采集5369条数据
已采集5370条数据
已采集5371条数据
已采集5372条数据
已采集5373条数据
已采集5374条数据
已采集5375条数据
已采集5376条数据
已采集5377条数据
已采集5378条数据
已采集5379条数据
已采集5380条数据
已采集5381条数据
已采集5382条数据
已采集5383条数据
已采集5384条数据
已采集5385条数据
已采集5386条数据
已采集5387条数据
已采集5388条数据
已采集5389条数据
已采集5390条数据
已采集5391条数据
已采集5392条数据
已采集5393条数据
已采集5394条数据
已采集5395条数据
已采集5396条数据
已采集5397条数据
已采集5398条数据
已采集5399条数据
已采集5400条数据

已采集6055条数据
已采集6056条数据
已采集6057条数据
已采集6058条数据
已采集6059条数据
已采集6060条数据
已采集6061条数据
已采集6062条数据
已采集6063条数据
已采集6064条数据
已采集6065条数据
已采集6066条数据
已采集6067条数据
已采集6068条数据
已采集6069条数据
已采集6070条数据
已采集6071条数据
已采集6072条数据
已采集6073条数据
已采集6074条数据
已采集6075条数据
已采集6076条数据
已采集6077条数据
已采集6078条数据
已采集6079条数据
已采集6080条数据
已采集6081条数据
已采集6082条数据
已采集6083条数据
已采集6084条数据
已采集6085条数据
已采集6086条数据
已采集6087条数据
已采集6088条数据
已采集6089条数据
已采集6090条数据
已采集6091条数据
已采集6092条数据
已采集6093条数据
已采集6094条数据
已采集6095条数据
已采集6096条数据
已采集6097条数据
已采集6098条数据
已采集6099条数据
已采集6100条数据
已采集6101条数据
已采集6102条数据
已采集6103条数据
已采集6104条数据
已采集6105条数据
已采集6106条数据
已采集6107条数据
已采集6108条数据
已采集6109条数据
已采集6110条数据
已采集6111条数据
已采集6112条数据
已采集6113条数据
已采集6114条数据
已采集6115条数据
已采集6116条数据
已采集6117条数据
已采集6118条数据
已采集6119条数据
已采集6120条数据
已采集6121条数据
已采集6122条数据
已采集6123条数据
已采集6124条数据
已采集6125条数据
已采集6126条数据
已采集6127条数据
已采集6128条数据
已采集6129条数据
已采集6130条数据
已采集6131条数据
已采集6132条数据
已采集6133条数据
已采集6134条数据
已采集6135条数据
已采集6136条数据
已采集6137条数据
已采集6138条数据
已采集6139条数据
已采集6140条数据
已采集6141条数据
已采集6142条数据
已采集6143条数据
已采集6144条数据
已采集6145条数据

已采集6800条数据
已采集6801条数据
已采集6802条数据
已采集6803条数据
已采集6804条数据
已采集6805条数据
已采集6806条数据
已采集6807条数据
已采集6808条数据
已采集6809条数据
已采集6810条数据
已采集6811条数据
已采集6812条数据
已采集6813条数据
已采集6814条数据
已采集6815条数据
已采集6816条数据
已采集6817条数据
已采集6818条数据
已采集6819条数据
已采集6820条数据
已采集6821条数据
已采集6822条数据
已采集6823条数据
已采集6824条数据
已采集6825条数据
已采集6826条数据
已采集6827条数据
已采集6828条数据
已采集6829条数据
已采集6830条数据
已采集6831条数据
已采集6832条数据
已采集6833条数据
已采集6834条数据
已采集6835条数据
已采集6836条数据
已采集6837条数据
已采集6838条数据
已采集6839条数据
已采集6840条数据
已采集6841条数据
已采集6842条数据
已采集6843条数据
已采集6844条数据
已采集6845条数据
已采集6846条数据
已采集6847条数据
已采集6848条数据
已采集6849条数据
已采集6850条数据
已采集6851条数据
已采集6852条数据
已采集6853条数据
已采集6854条数据
已采集6855条数据
已采集6856条数据
已采集6857条数据
已采集6858条数据
已采集6859条数据
已采集6860条数据
已采集6861条数据
已采集6862条数据
已采集6863条数据
已采集6864条数据
已采集6865条数据
已采集6866条数据
已采集6867条数据
已采集6868条数据
已采集6869条数据
已采集6870条数据
已采集6871条数据
已采集6872条数据
已采集6873条数据
已采集6874条数据
已采集6875条数据
已采集6876条数据
已采集6877条数据
已采集6878条数据
已采集6879条数据
已采集6880条数据
已采集6881条数据
已采集6882条数据
已采集6883条数据
已采集6884条数据
已采集6885条数据
已采集6886条数据
已采集6887条数据
已采集6888条数据
已采集6889条数据
已采集6890条数据

## 将结果保存为本地文件

In [45]:
df = pd.DataFrame(data)
df.head()

,房源链接,总价,单价,挂牌价格,成交周期,房屋户型,所在楼层,建筑面积,户型结构,套内面积,建筑类型,房屋朝向,建成年代,装修情况,建筑结构,供暖方式,梯户比例,配备电梯,链家编号,交易权属,挂牌时间,房屋用途,房屋年限,房权所属
0,https://xm.lianjia.com/chengjiao/105104776786....,40万,10858元/平,40挂牌价格（万）,66成交周期（天）,车位,地下室(共1层),36.84㎡,暂无数据,暂无数据,板楼,南,2015,简装,钢结构,,暂无数据,无,105104776786,商品房,2020-07-07,车库,暂无数据,非共有
1,https://xm.lianjia.com/chengjiao/105104748596....,675万,72193元/平,695挂牌价格（万）,69成交周期（天）,2室2厅1厨1卫,低楼层(共26层),93.5㎡,平层,暂无数据,板楼,东南,2015,精装,钢混结构,,三梯四户,有,105104748596,商品房,2020-07-04,普通住宅,暂无数据,非共有
2,https://xm.lianjia.com/chengjiao/105105262186....,905万,63054元/平,930挂牌价格（万）,5成交周期（天）,3室2厅1厨2卫,中楼层(共10层),143.53㎡,平层,暂无数据,板楼,东 南 北,2000,简装,钢混结构,,一梯两户,有,105105262186,商品房,2020-09-06,普通住宅,暂无数据,共有
3,https://xm.lianjia.com/chengjiao/105104387813....,370万,59362元/平,370挂牌价格（万）,115成交周期（天）,2室1厅1厨1卫,中楼层(共6层),62.33㎡,平层,暂无数据,板楼,南 北,1989,精装,砖混结构,,一梯两户,无,105104387813,商品房,2020-05-19,普通住宅,暂无数据,非共有
4,https://xm.lianjia.com/chengjiao/105105203556....,318万,62834元/平,330挂牌价格（万）,12成交周期（天）,2室1厅1厨1卫,高楼层(共6层),50.61㎡,平层,48㎡,塔楼,南,1994,简装,钢混结构,,一梯两户,无,105105203556,商品房,2020-08-30,普通住宅,满五年,非共有
5,https://xm.lianjia.com/chengjiao/105105079159....,311万,59831元/平,328挂牌价格（万）,27成交周期（天）,2室1厅1厨1卫,低楼层(共6层),51.98㎡,平层,暂无数据,板楼,南 北,1988,精装,砖混结构,,一梯两户,无,105105079159,房改房,2020-08-14,普通住宅,暂无数据,非共有
6,https://xm.lianjia.com/chengjiao/105104090087....,1286万,47676元/平,1320挂牌价格（万）,149成交周期（天）,4室1厅1厨4卫,中楼层(共29层),269.74㎡,平层,暂无数据,板楼,南,2011,毛坯,钢混结构,,两梯两户,有,105104090087,商品房,2020-04-14,普通住宅,满五年,非共有
7,https://xm.lianjia.com/chengjiao/105104430513....,355万,30521元/平,368挂牌价格（万）,109成交周期（天）,3室2厅1厨2卫,高楼层(共25层),116.25㎡,暂无数据,暂无数据,塔楼,南,未知,精装,框架结构,,七梯十二户,有,105104430513,商品房,2020-05-24,商业办公类,暂无数据,共有
8,https://xm.lianjia.com/chengjiao/XMHL92676034....,900万,63039元/平,900挂牌价格（万）,24成交周期（天）,3室2厅1厨2卫,低楼层(共27层),142.77㎡,平层,暂无数据,板塔结合,南 北,2012,精装,钢混结构,,两梯两户,有,XMHL92676034,商品房,2016-09-06,普通住宅,满两年,暂无数据
9,https://xm.lianjia.com/chengjiao/XMHL92095560....,634万,48890元/平,605挂牌价格（万）,8成交周期（天）,3室2厅1厨2卫,低楼层(共27层),129.68㎡,暂无数据,暂无数据,板楼,南,2013,简装,钢混结构,,两梯两户,有,XMHL92095560,商品房,2016-04-29,普通住宅,暂无数据,暂无数据


In [46]:
# 保存为本地Excel文件，文件名包含采集时间
df.to_excel(r"厦门链家网成交二手房爬虫数据_{}.xlsx".format(date.today()))